In [1]:
import os
import torch
import torchvision
from torch import nn, optim
from torchvision import transforms, models, datasets
!pip install torchsummary
from torchsummary import summary
from PIL import Image
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('/kaggle/input/bizinnovate-2023/train.csv')
train['path'] = train['path'].str.replace('dhs_train/', '')
train.head()

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1dhs,urban,path
0,IA-2015-7-00010004,IA,2015,9.165413,92.742696,22.0,2.650768,22.0,5.000000,10004,1,R,IA-2015-7-00010004.npz
1,IA-2015-7-00010005,IA,2015,8.307356,93.093792,22.0,2.157784,22.0,5.000000,10005,1,R,IA-2015-7-00010005.npz
2,IA-2015-7-00010007,IA,2015,7.016968,93.893226,21.0,1.832751,21.0,4.619048,10007,1,R,IA-2015-7-00010007.npz
3,IA-2015-7-00010016,IA,2015,9.194938,92.800432,22.0,2.746096,22.0,5.000000,10016,1,R,IA-2015-7-00010016.npz
4,IA-2015-7-00010018,IA,2015,8.055606,93.543892,22.0,2.581869,22.0,5.000000,10018,1,R,IA-2015-7-00010018.npz


In [3]:
test = pd.read_csv('/kaggle/input/bizinnovate-2023/test_masked.csv')
test['path'] = test['path'].str.replace('dhs_valid/', '')
test.head()

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,cluster_id,adm1dhs,urban,path
0,IA-2015-7-00010009,IA,2015,9.220903,92.781530,22.0,2.721812,10009,1,R,IA-2015-7-00010009.npz
1,IA-2015-7-00010011,IA,2015,7.028410,93.883430,20.0,2.287279,10011,1,R,IA-2015-7-00010011.npz
2,IA-2015-7-00010017,IA,2015,12.371448,92.783665,22.0,0.677109,10017,1,R,IA-2015-7-00010017.npz
3,IA-2015-7-00010044,IA,2015,11.727304,92.719257,21.0,1.793683,10044,1,R,IA-2015-7-00010044.npz
4,IA-2015-7-00010060,IA,2015,9.185310,92.777645,22.0,2.758168,10060,1,R,IA-2015-7-00010060.npz


In [4]:
corr = train.corr()
corr.style.background_gradient(cmap='coolwarm')

/opt/conda/lib/python3.7/site-packages/pandas/io/formats/style.py:2813: RuntimeWarning: All-NaN slice encountered
  smin = np.nanmin(gmap) if vmin is None else vmin
/opt/conda/lib/python3.7/site-packages/pandas/io/formats/style.py:2814: RuntimeWarning: All-NaN slice encountered
  smax = np.nanmax(gmap) if vmax is None else vmax


,year,lat,lon,n_asset,asset_index,n_water,water_index,cluster_id,adm1dhs
year,nan,nan,nan,nan,nan,nan,nan,nan,nan
lat,nan,1.000000,0.153961,0.003819,-0.137063,0.003819,-0.234538,-0.002257,-0.004206
lon,nan,0.153961,1.000000,-0.001532,-0.265972,-0.001532,-0.324953,-0.155466,-0.154863
n_asset,nan,0.003819,-0.001532,1.000000,-0.181002,1.000000,-0.138688,-0.032996,-0.033262
asset_index,nan,-0.137063,-0.265972,-0.181002,1.000000,-0.181002,0.723632,0.026653,0.029448
n_water,nan,0.003819,-0.001532,1.000000,-0.181002,1.000000,-0.138688,-0.032996,-0.033262
water_index,nan,-0.234538,-0.324953,-0.138688,0.723632,-0.138688,1.000000,-0.013246,-0.010455
cluster_id,nan,-0.002257,-0.155466,-0.032996,0.026653,-0.032996,-0.013246,1.000000,0.999942
adm1dhs,nan,-0.004206,-0.154863,-0.033262,0.029448,-0.033262,-0.010455,0.999942,1.000000


In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device.type}")

Using device: cuda


In [6]:
class NPZDataset(Dataset):
    def __init__(self, path, csv, transform=None):
        self.path = path
        self.water_indexes = pd.read_csv(csv)
        self.files = list(Path(path).glob('*.npz'))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        numpy_array = np.load(str(self.files[idx]))['x']
        sat_img = np.moveaxis(numpy_array, 0, -1)[:, :, 2:5].transpose((-1, 0, 1))
        torch_array = torch.from_numpy(sat_img)
        if self.transform is not None:
            torch_array = self.transform(torch_array)
        y = self.water_indexes.iloc[idx, 8]
        y = torch.tensor(y, dtype=torch.float64)
        return torch_array.cuda(), y.cuda()

In [7]:
train_dir = "/kaggle/input/bizinnovate-2023/dhs_train"
train_y = "/kaggle/input/bizinnovate-2023/train.csv"
valid_dir = "/kaggle/input/bizinnovate-2023/dhs_test"

transform = transforms.Compose([
        transforms.Normalize(torch.tensor([0.1198, 0.2080, 0.1475]), torch.tensor([0.0219, 0.0451, 0.0345]))
])

train_data = NPZDataset(train_dir, train_y, transform=transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

# valid_data = NPZDataset(valid_dir)
# valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)

In [8]:
imgs, labs = next(iter(train_loader))

In [9]:
imgs.shape

torch.Size([64, 3, 255, 255])

In [10]:
labs.shape

torch.Size([64])

In [11]:
# from torchvision import utils

# plt.figure(figsize=(10, 8))
# plt.axis("off")
# plt.title("Sample Training Images")
# plt.imshow(np.transpose(utils.make_grid(imgs, padding=1, normalize=True),(1,2,0)));

In [12]:
def plot_convs(image, conv_layer, axis=False):
    """Plot convs with matplotlib. Sorry for this lazy code :D"""
    filtered_image = conv_layer(image[None, None, :])
    n = filtered_image.shape[1]
    if n == 1:
        fig, (ax1, ax2) = plt.subplots(figsize=(8, 4), ncols=2)
        ax1.imshow(image, cmap='gray')
        ax1.set_title("Original")
        ax2.imshow(filtered_image.detach().squeeze(), cmap='gray')  
        ax2.set_title("Filter 1")
        ax1.grid(False)
        ax2.grid(False)
        if not axis:
            ax1.axis(False)
            ax2.axis(False)
        plt.tight_layout();
    elif n == 2:
        filtered_image_1 = filtered_image[:,0,:,:]
        filtered_image_2 = filtered_image[:,1,:,:]
        fig, (ax1, ax2, ax3) = plt.subplots(figsize=(10, 4), ncols=3)
        ax1.imshow(image, cmap='gray')
        ax1.set_title("Original")
        ax2.imshow(filtered_image_1.detach().squeeze(), cmap='gray')  
        ax2.set_title("Filter 1")
        ax3.imshow(filtered_image_2.detach().squeeze(), cmap='gray')  
        ax3.set_title("Filter 2")
        ax1.grid(False)
        ax2.grid(False)
        ax3.grid(False)
        if not axis:
            ax1.axis(False)
            ax2.axis(False)
            ax3.axis(False)
        plt.tight_layout();
    elif n == 3:
        filtered_image_1 = filtered_image[:,0,:,:]
        filtered_image_2 = filtered_image[:,1,:,:]
        filtered_image_3 = filtered_image[:,2,:,:]
        fig, (ax1, ax2, ax3, ax4) = plt.subplots(figsize=(12, 4), ncols=4)
        ax1.imshow(image, cmap='gray')
        ax1.set_title("Original")
        ax2.imshow(filtered_image_1.detach().squeeze(), cmap='gray')  
        ax2.set_title("Filter 1")
        ax3.imshow(filtered_image_2.detach().squeeze(), cmap='gray')  
        ax3.set_title("Filter 2")
        ax4.imshow(filtered_image_3.detach().squeeze(), cmap='gray')  
        ax4.set_title("Filter 3")
        ax1.grid(False)
        ax2.grid(False)
        ax3.grid(False)
        ax4.grid(False)
        if not axis:
            ax1.axis(False)
            ax2.axis(False)
            ax3.axis(False)
            ax4.axis(False)
        plt.tight_layout();
    
# conv_layer = torch.nn.Conv2d(1, 1, kernel_size=(3, 3))
# plot_convs(imgs[0].squeeze().detach(), conv_layer)

In [13]:
# conv_layer = torch.nn.Conv2d(1, 2, kernel_size=(3, 3))
# plot_convs(imgs[0].squeeze().detach(), conv_layer)

In [14]:
# conv_layer = torch.nn.Conv2d(1, 3, kernel_size=(5, 5))
# plot_convs(imgs[0].squeeze().detach(), conv_layer)

In [15]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3, 8, (5, 5)),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),
            nn.Dropout(0.2),

            nn.Conv2d(8, 4, (3, 3)),
            nn.ReLU(),
            nn.MaxPool2d((3, 3)),
            nn.Dropout(0.2),

            nn.Flatten(),
            nn.Linear(6724, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )
        
    def forward(self, x):
        out = self.main(x)
        return out

In [16]:
def trainer(model, criterion, optimizer, dataloader, epochs=5, verbose=True):
    """Simple training wrapper for PyTorch network."""
    
    train_loss = []
    
    for epoch in range(epochs):  # for each epoch
        losses = 0
        
        for X, y in dataloader:  # for each batch
            X = X.float()
            y = y.float()
            optimizer.zero_grad()            # Zero all the gradients w.r.t. parameters
            
            y_hat = model(X).flatten()       # Forward pass to get output
            loss = criterion(y_hat, y)       # Calculate loss based on output
            loss.backward()                  # Calculate gradients w.r.t. parameters
            optimizer.step()                 # Update parameters

            losses += loss.item()            # Add loss for this batch to running total
        
        if verbose:
            print(f"epoch: {epoch + 1}, loss: {losses / len(dataloader):.4f}")
            
        train_loss.append(losses / len(dataloader))  # loss = total loss in epoch / number of batches = loss per batch
        
    return train_loss

In [17]:
model = CNN()
model.to(device);
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)
train_loss = trainer(model, criterion, optimizer, train_loader, epochs=100)

epoch: 1, loss: 1.0315


epoch: 2, loss: 0.7660


epoch: 3, loss: 0.7602


epoch: 4, loss: 0.7625


epoch: 5, loss: 0.7543


...

epoch: 94, loss: 0.6155


epoch: 95, loss: 0.6124


epoch: 96, loss: 0.6092


epoch: 97, loss: 0.6015


epoch: 98, loss: 0.6069


In [ ]:
class TestNPZDataset(Dataset):
    def __init__(self, path, transform=None):
        self.path = path
        self.files = list(Path(path).glob('*.npz'))
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        numpy_array = np.load(str(self.files[idx]))['x']
        sat_img = np.moveaxis(numpy_array, 0, -1)[:, :, 2:5].transpose((-1, 0, 1))
        torch_array = torch.from_numpy(sat_img)
        if self.transform is not None:
            torch_array = self.transform(torch_array)
        return torch_array.cuda()

In [ ]:
valid_data = TestNPZDataset(valid_dir, transform=transform)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=True)

In [ ]:
def predict(model, dataloader):
    preds = []
    for X in dataloader:
        output = model(X)
        preds.append(output)
    return preds

In [ ]:
predictions = predict(model, valid_loader)
test['water_index'] = predictions
submission = test[['DHSID_EA', 'water_index']]
submission.to_csv('test.csv', index=False)

In [ ]:
# class AE(nn.Module):
#     def __init__(self, **kwargs):
#         super().__init__()
#         self.encoder_hidden_layer = nn.Linear(
#             in_features=kwargs["input_shape"], out_features=128
#         )
#         self.encoder_output_layer = nn.Linear(
#             in_features=128, out_features=128
#         )
#         self.decoder_hidden_layer = nn.Linear(
#             in_features=128, out_features=128
#         )
#         self.decoder_output_layer = nn.Linear(
#             in_features=128, out_features=kwargs["input_shape"]
#         )

#     def forward(self, features):
#         activation = self.encoder_hidden_layer(features)
#         activation = torch.relu(activation)
#         code = self.encoder_output_layer(activation)
#         code = torch.relu(code)
#         activation = self.decoder_hidden_layer(code)
#         activation = torch.relu(activation)
#         activation = self.decoder_output_layer(activation)
#         reconstructed = torch.relu(activation)
#         return reconstructed

In [ ]:

# #  use gpu if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # create a model from `AE` autoencoder class
# # load it to the specified device, either gpu or cpu
# model = AE(input_shape=784).to(device)

# # create an optimizer object
# # Adam optimizer with learning rate 1e-3
# optimizer = optim.Adam(model.parameters(), lr=1e-3)

# # mean-squared error loss
# criterion = nn.MSELoss()